# 6. 멀티인덱스

groupby() 메소드에 여러 열을 리스트 형태로 전달하면 각 열들이 다중으로 행 인덱스를 구성 <br>
판다스는 행 인덱스를 여러 레벨로 구현할 수 있도록 멀티 인덱스 클래스를 지원

In [3]:
# 라이브러리 및 데이터 불러오기
import pandas as pd
import seaborn as sns

titanic = sns.load_dataset('titanic')
df = titanic.loc[:, ['age', 'sex', 'class', 'fare', 'survived']]
print(df)

      age     sex   class     fare  survived
0    22.0    male   Third   7.2500         0
1    38.0  female   First  71.2833         1
2    26.0  female   Third   7.9250         1
3    35.0  female   First  53.1000         1
4    35.0    male   Third   8.0500         0
..    ...     ...     ...      ...       ...
886  27.0    male  Second  13.0000         0
887  19.0  female   First  30.0000         1
888   NaN  female   Third  23.4500         0
889  26.0    male   First  30.0000         1
890  32.0    male   Third   7.7500         0

[891 rows x 5 columns]


In [5]:
# 객실 등급(class)과 남녀 성별(sex)을 기준으로 그룹화
grouped = df.groupby(['class', 'sex'])

# 그룹 객체에 연산 메소드를 적용
# 즉, 각 그룹별로 각 열의 평균값을 정리하여 반환
gdf = grouped.mean()
print(gdf, '\n')
print(type(gdf), '\n')

                     age        fare  survived
class  sex                                    
First  female  34.611765  106.125798  0.968085
       male    41.281386   67.226127  0.368852
Second female  28.722973   21.970121  0.921053
       male    30.740707   19.741782  0.157407
Third  female  21.750000   16.118810  0.500000
       male    26.507589   12.661633  0.135447 

<class 'pandas.core.frame.DataFrame'> 



## 멀티인덱스에서 특정 인덱스를 뽑아내는 방법
* loc 인덱서 : 두 개의 인덱스를 이용할 경우 튜플 형태로 전달
* xs 인덱서 : level 옵션을 이용

In [8]:
# 1. loc 인덱서

# class 값이 First인 행을 선택하여 출력
print(gdf.loc['First'], '\n')

# class 값이 First이고, sex의 값이 female인 행을 선택하여 출력
print(gdf.loc[('First', 'female')], '\n')

              age        fare  survived
sex                                    
female  34.611765  106.125798  0.968085
male    41.281386   67.226127  0.368852 

age          34.611765
fare        106.125798
survived      0.968085
Name: (First, female), dtype: float64 



In [9]:
# 2. xs 인덱서

# sex의 값이 male인 행을 선택하여 출력
print(gdf.xs('male', level = 'sex'))

              age       fare  survived
class                                 
First   41.281386  67.226127  0.368852
Second  30.740707  19.741782  0.157407
Third   26.507589  12.661633  0.135447


---

# 7. 피벗
* 엑셀에서 사용하는 피벗데이블과 비슷한 기능을 처리
* 판다스 pivot_table() 함수 이용
* 피벗테이블을 구성하는 4가지 요소 
 1. 행 인덱스(index)
 2. 열 인덱스(columns)
 3. 데이터 값(values)
 4. 데이터 집계 함수(aggfunc)
* 4가지 요소에 적용할 데이터프레임의 열을 각각 지정하여 함수의 인자로 전달

In [11]:
# 라이브러리 및 데이터 불러오기
import pandas as pd
import seaborn as sns

pd.set_option('display.max_columns', 10)
pd.set_option('display.max_colwidth', 20)

titanic = sns.load_dataset('titanic')
df = titanic.loc[:, ['age', 'sex', 'class', 'fare', 'survived']]
print(df.head())

    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
1  38.0  female  First  71.2833         1
2  26.0  female  Third   7.9250         1
3  35.0  female  First  53.1000         1
4  35.0    male  Third   8.0500         0


In [14]:
# 행, 열, 값, 집계에 사용할 열을 1개씩 지정 - 평균 집계
pdf1 = pd.pivot_table(df,
                      index = 'class',
                      columns = 'sex',
                      values = 'age',
                      aggfunc = 'mean')
print(pdf1.head())

sex        female       male
class                       
First   34.611765  41.281386
Second  28.722973  30.740707
Third   21.750000  26.507589


In [15]:
# 값에 적용하는 집계 함수 2개 이상 지정 가능 - 생존율, 생존자 수 집계
# 열 구조가 2중 멀티 인덱스가 됨
# 즉, 데이터 집계 함수가 한 층을 이루고 열이 한 층을 이룸
pdf2 = pd.pivot_table(df,
                      index = 'class',
                      columns = 'sex',
                      values = 'survived',
                      aggfunc = ['mean', 'sum'])
print(pdf2.head())

            mean              sum     
sex       female      male female male
class                                 
First   0.968085  0.368852     91   45
Second  0.921053  0.157407     70   17
Third   0.500000  0.135447     72   47


In [19]:
# 행, 열, 값에 사용할 열을 2개 이상 지정 가능 - 평균 나이, 최대 요금 집계
pdf3 = pd.pivot_table(df,
                      index = ['class', 'sex'],
                      columns = 'survived',
                      values = ['age', 'fare'],
                      aggfunc = ['mean', 'max'])
print(pdf3.head(), '\n')
print(pdf3.index, '\n') 
print(pdf3.columns, '\n')

                    mean                                      max        \
                     age                   fare               age         
survived               0          1           0           1     0     1   
class  sex                                                                
First  female  25.666667  34.939024  110.604167  105.978159  50.0  63.0   
       male    44.581967  36.248000   62.894910   74.637320  71.0  80.0   
Second female  36.000000  28.080882   18.250000   22.288989  57.0  55.0   
       male    33.369048  16.022000   19.488965   21.095100  70.0  62.0   
Third  female  23.818182  19.329787   19.773093   12.464526  48.0  63.0   

                                 
                 fare            
survived            0         1  
class  sex                       
First  female  151.55  512.3292  
       male    263.00  512.3292  
Second female   26.00   65.0000  
       male     73.50   39.0000  
Third  female   69.55   31.3875   

MultiIndex([( 'F

## 멀티인덱스에서 특정 인덱스 뽑아내기
* xs 인덱서 : 행 선택(axis = 0, 기본값), 열 선택(axis = 1)

In [25]:
# 행 선택 (기본값 : axis = 0)
print(' * 레벨 0에서 행 선택')
print(pdf3.xs('First'), '\n')

# 행 인덱스 레벨 0에서 First, 레벌 1에서 female 가져오기
# 2개의 인덱스 값을 튜플로 전달
print(' * 레벌 0, 1에서 행 선택')
print(pdf3.xs(('First', 'female')), '\n')

# 행 인덱스 레벨 직접 지정
print(' * 레벨 직접 지정(1)')
print(pdf3.xs('male', level = 'sex'), '\n')

print(' * 레벨 직접 지정(2)')
print(pdf3.xs(('Second', 'male'), level = [0, 1])) # level = ['class', 'sex']와 동일

 * 레벨 0에서 행 선택
               mean                                      max                \
                age                   fare               age          fare   
survived          0          1           0           1     0     1       0   
sex                                                                          
female    25.666667  34.939024  110.604167  105.978159  50.0  63.0  151.55   
male      44.581967  36.248000   62.894910   74.637320  71.0  80.0  263.00   

                    
                    
survived         1  
sex                 
female    512.3292  
male      512.3292   

 * 레벌 0, 1에서 행 선택
            survived
mean  age   0            25.666667
            1            34.939024
      fare  0           110.604167
            1           105.978159
max   age   0            50.000000
            1            63.000000
      fare  0           151.550000
            1           512.329200
Name: (First, female), dtype: float64 

 * 레벨 직접 지정(1)
              

In [29]:
# 열 선택(axis = 1)
print('* 열 선택 (1)')
print(pdf3.xs('mean', axis = 1), '\n')

# 열 인덱스 레벨 0에서 mean 지정, 레벨 1에서 age 지정
print('* 열 선택 (2)')
print(pdf3.xs(('mean', 'age'), axis = 1), '\n')

# 인덱스 레벨 직접 지정
print('* 인덱스 레벨 직접 지정')
print(pdf3.xs(1, level = 'survived', axis = 1), '\n')

# 3개의 열 인덱스 모두 지정
print('* 3개의 열 인덱스 모두 지정')
print(pdf3.xs(('max', 'fare', 0), level = [0, 1, 2], axis = 1))

* 열 선택 (1)
                     age                   fare            
survived               0          1           0           1
class  sex                                                 
First  female  25.666667  34.939024  110.604167  105.978159
       male    44.581967  36.248000   62.894910   74.637320
Second female  36.000000  28.080882   18.250000   22.288989
       male    33.369048  16.022000   19.488965   21.095100
Third  female  23.818182  19.329787   19.773093   12.464526
       male    27.255814  22.274211   12.204469   15.579696 

* 열 선택 (2)
survived               0          1
class  sex                         
First  female  25.666667  34.939024
       male    44.581967  36.248000
Second female  36.000000  28.080882
       male    33.369048  16.022000
Third  female  23.818182  19.329787
       male    27.255814  22.274211 

* 인덱스 레벨 직접 지정
                    mean               max          
                     age        fare   age      fare
class  sex               